In [11]:
from pathlib import Path

from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.ai.ml import load_component
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline


In [2]:
credential = DefaultAzureCredential()
credential.get_token("https://management.azure.com/.default")

ml_client = MLClient.from_config(credential=credential)


Found the config file in: /config.json


In [77]:
!source prep-srcs.sh


In [78]:
component_yaml_dir = Path("src")

data_prep = load_component(source = component_yaml_dir / "data_prep.yml")
train_kernelridge = load_component(source = component_yaml_dir / "train_kernelridge.yml")


In [79]:
@pipeline()
def house_prices_pipeline(train_data, test_data, data_description, kernel):
    data_prep_result = data_prep(train_data = train_data,
                           test_data = test_data,
                           data_description = data_description)

    train_kernelridge_result = train_kernelridge(prep_data_dir = data_prep_result.outputs.output_data_dir, kernel = kernel)

    return {
        "pipeline_job_data_prep_output_dir": data_prep_result.outputs.output_data_dir,
        "pipeline_job_sample_submission": train_kernelridge_result.outputs.output_dir,
    }


In [80]:
# pipeline_job = house_prices_pipeline()

train_data = Input(type=AssetTypes.URI_FILE, path="azureml:training-data:1")
test_data = Input(type=AssetTypes.URI_FILE, path="azureml:test-data:1")
data_description = Input(type=AssetTypes.URI_FILE, path="azureml:data-desc:1")
kernel = 'linear'

pipeline_job = house_prices_pipeline(train_data = train_data,
                                     test_data = test_data,
                                     data_description = data_description,
                                     kernel = kernel)


In [81]:
pipeline_job.outputs.pipeline_job_sample_submission.mode = 'upload'


In [82]:
pipeline_job.settings.default_compute = "aml-cluster"
pipeline_job.settings.default_datastore = "workspaceblobstore"


In [83]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline_house_prices"
)
pipeline_job


Uploading train_kernelridge_src (0.03 MBs): 100%|██████████| 25195/25195 [00:00<00:00, 696085.27it/s]


pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored


Experiment,Name,Type,Status,Details Page
pipeline_house_prices,cool_parcel_p65xnr1j3t,pipeline,NotStarted,Link to Azure Machine Learning studio
